In [9]:
import openai
import json

#model = "gpt-3.5-turbo-0613"
model = "gpt-4-0613"

def ask_user(question):
    return input(question)

functions = [
    {
      "name": "ask_user",
      "description": "Asks the user for input",
      "parameters": {
        "type": "object",
        "properties": {
          "question": {
            "type": "string",
            "description": "The question to ask to the user"
          },
        },
        "required": ["question"]
      }
    },
  ]

available_functions = {
    "ask_user": ask_user,
}

In [10]:
def run_conversation(content):
  messages = [
    {"role": "user", "content": content},
    ]
  # Step 1: send the conversation and available functions to GPT
  response = openai.ChatCompletion.create(
    model=model,
    messages=messages,
    functions=functions
  )
  response_message = response["choices"][0]["message"]
  messages.append(response_message)  # extend conversation with assistant's reply

  # Step 2: check if GPT wanted to call a function
  while response_message.get("function_call"):
    # Step 3: call the function
    # Note: the JSON response may not always be valid; be sure to handle errors
    function_name = response_message["function_call"]["name"]
    if function_name not in available_functions:
      raise Exception("Function " + function_name + " not found")
    function_to_call = available_functions[function_name]
    function_args = json.loads(response_message["function_call"]["arguments"])
    function_response = function_to_call(**function_args)

    # Step 4: send the info on the function call and function response to GPT
    messages.append(
        {
            "role": "function",
            "name": function_name,
            "content": function_response,
        }
    )  # extend conversation with function response
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
    )  # get a new response from GPT where it can see the function response
    response_message = response["choices"][0]["message"]
    messages.append(response_message)  # extend conversation with assistant's reply

  for message in messages:
    if message.get("content"):
      print(message["role"] + ": " + message["content"])
    if message.get("function_call"):
      print(" -> calling function " + message["function_call"]["name"] + " with arguments " + json.dumps(message["function_call"]["arguments"]))
  

In [12]:
run_conversation("""If my brother is 3 years younger than me, how old is my brother?
    Ask user for any missing information.
    Explain step by step.""")

user: If my brother is 3 years younger than me, how old is my brother?
    Ask user for any missing information.
    Explain step by step.
 -> calling function ask_user with arguments "{\n  \"question\": \"How old are you?\"\n}"
function: 35
assistant: Since your brother is 3 years younger than you, and you are 35, we will subtract 3 from 35 to find out your brother's age.

Step by step explanation:
1. The age difference between you and your brother is 3 years.
2. Your current age is 35 years.
3. To find out your brother's age, we need to subtract the age difference from your age.
4. In other words, your brother's age would be 35 years - 3 years = 32 years.

So, your brother is 32 years old.
